# Prompt Scoring & Evaluation v1

This notebook explored updates to the initial prompt based on initial findings from testing extension and demo app. It reuses the initial prompt testing setup (Prompt A vs B) with a detailed evaluation framework to assess Easy Language compliance.

**Goals:**
1. Load English samples.
2. Run Prompt A (Baseline) and Prompt B (Experimental).
3. Evaluate both outputs against specific Easy Language rules.
4. Display side-by-side comparison with scores.

# Prompt conventions

The prompts are defined as follows:

- Prompt 0 (Naive) - minimal to show model performance with little to no guidance (this is what Prompt A was evaluated against)
- Prompt A (Baseline) - reference final version to start our journey. Goal is to evaluate against this.
- Prompt B (Experimental) - the one being evaluated

Testing will use the same defined test set

In this notebook we will look at the following:
1) System prompt + no invention
2) Review chunking
3) Output formatting rules: ex. overuse of bullet points
4) Add post-generation guardrails (deterministic checks)

Additional details on each of these can be found in apps/Improvement_Strategy.md

## 1. Setup & Imports

In [16]:
import os
import re
import time
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from groq import Groq
from pybars import Compiler
from IPython.display import display, HTML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load .env
load_dotenv(find_dotenv(usecwd=True))

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY environment variable not set.")

GROQ_MODEL = "llama-3.1-8b-instant" 
# GROQ_MODEL = "llama-3.3-70b-versatile" # Optional: switch to larger model

try:
    client = Groq(api_key=GROQ_API_KEY)
    print(f"✅ Setup complete. Using model: {GROQ_MODEL}")
except Exception as e:
    print(f"❌ Error initializing Groq client: {e}")

✅ Setup complete. Using model: llama-3.1-8b-instant


## 2. Evaluation Helper Functions
For v1 this is revised and new rule added to test specific issues identified in the Stategy doc. The revision moves that checks of the easy language rules from evaluating intrinsic rules (output only) to comparative (output vs original). Keeping the tfidf calculation. This is also easier on the API.

Additional helper functions added and bullet use revised to "appropriate_bullet_use" because we know we want bullets just not overdone.

In [17]:
def tfidf_similarity(text1: str, text2: str) -> float:
    """Calculate TF-IDF cosine similarity between two texts."""
    try:
        vectorizer = TfidfVectorizer(lowercase=True, stop_words=None)
        tfidf_matrix = vectorizer.fit_transform([text1, text2])
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        return round(similarity, 3)
    except:
        return 0.0

# Easy Language Rules Definition
EASY_LANGUAGE_RULES = {
    "short_sentences": {
        "name": "Short Sentences",
        "description": "Max 15% of sentences > 10 words",
        "check": lambda output, original: (
            sum(1 for s in re.split(r'[.!?\n]', output) 
                if s.strip() and len(s.split()) > 10) / 
            max(1, len([s for s in re.split(r'[.!?\n]', output) if s.strip()]))
        ) * 100
    },
     "appropriate_bullet_use": {
        "name": "Appropriate Bullet Points",
        "description": "Uses bullet points or numbered lists for steps, lists or multiple items",
        "check": lambda output, original: check_bullet_appropriateness(output, original)
    },
    "has_paragraphs": {
        "name": "Clear Paragraphs",
        "description": "Has blank lines between sections",
        "check": lambda output, original: '\n\n' in output or '\n \n' in output
    },
    "no_intro_text": {
        "name": "No Intro/Outro Text",
        "description": "No introductory or concluding text at the start",
        "check": lambda output, original: not bool(
            re.search(r'^(Here\'s|Here is|This is|The following|Hier ist|In summary|To summarize|You want|Let me)', 
                    output.strip()[:200], re.IGNORECASE)
        )
    },
    "no_xml_tags": {
        "name": "No XML/HTML Tags",
        "description": "Never output any XML/HTML tags or attributes (no <...>, no id=...)",
        "check": lambda output, original: not bool(re.search(r'<[^>]+>|id\s*=', output))
    },
    "keep_meaning": {
        "name": "Keep Meaning",
        "description": "Do not drop meaning - rewrite sentence by sentence, do not condense or join",
        "check": lambda output, original: tfidf_similarity(original, output)  # Manual review needed / Used with TF-IDF
    },
    "active_voice": {
        "name": "Active Voice",
        "description": "Uses active voice (approximation: few passive markers)",
        "check": lambda output, original: (
            output.lower().count(' is ') + 
            output.lower().count(' are ') + 
            output.lower().count(' was ') + 
            output.lower().count(' were ')
        ) < 5
    },
# New rules for v1
    "preserves_proper_nouns": {
    "name": "Preserves Proper Nouns",
    "description": "All names, products, organizations preserved exactly",
    "check": lambda output, original: check_proper_nouns(output, original)  # New function needed
    },
    "no_temporal_injection": {
        "name": "No Temporal Injection",
        "description": "Doesn't add 'today', 'now', 'currently' unless in source",
        "check": lambda output, original: (
            not bool(re.search(r'\b(today|now|currently)\b', output, re.I)) or 
            bool(re.search(r'\b(today|now|currently)\b', original, re.I))
        )
    },
    "preserves_numbers_dates": {
        "name": "Preserves Numbers & Dates",
        "description": "All numbers and dates from source appear in output",
        "check": lambda output, original: check_numbers_preserved(output, original)  # New function needed
    },
    "no_heading_expansion": {
        "name": "No Heading Expansion",
        "description": "Short titles stay as titles, not expanded to bullets",
     "check": lambda text, original: not (is_heading(original) and has_bullets(text))  # New function needed
    }
}

def evaluate_rules(text: str, original_text: str = None) -> dict:
    """
    Check how well the text follows Easy Language rules.
    
    Args:
        text: The simplified output text to evaluate
        original_text: The original source text for comparison
        
    Returns:
        Dictionary mapping rule_id to {"value": <result>, "pass": <bool>}
    """
    results = {}
    
    # Use empty string if no original provided (for rules that need it)
    original = original_text if original_text else ""
    
    for rule_id, rule in EASY_LANGUAGE_RULES.items():
        # Call the rule's check function with consistent signature
        check_result = rule["check"](text, original)
        
        # Determine pass/fail based on rule type
        if rule_id == "short_sentences":
            # Percentage check: pass if <= 15%
            results[rule_id] = {
                "value": check_result, 
                "pass": check_result <= 15
            }
        
        elif rule_id == "keep_meaning":
            # Similarity check: pass if >= 0.3 (30% similarity)
            results[rule_id] = {
                "value": check_result,
                "pass": check_result >= 0.3 if original_text else True
            }
        
        else:
            # Boolean checks: True = pass
            results[rule_id] = {
                "value": check_result,
                "pass": bool(check_result)
            }
    
    return results

In updating to comparative logic, some of the rules have complex multi-step logic. This helps testing and readability. These are extracted to separate functions if they: 
1) have more that 2 operations or nested logic
2) are used in multiple places

In [18]:
#this cell defines evaluation functions 

def check_proper_nouns(text: str, original: str) -> bool:
    """Check if proper nouns are preserved - enhanced to capture acronyms, camelCase, and tokens with digits."""
    must_keep = set()
    
    # Capitalized sequences including acronyms (2+ consecutive caps like WCAG, WAI, API)
    must_keep.update(re.findall(r'\b[A-Z]{2,}[a-z]*\b', original))
    
    # CamelCase tokens (like OpenAI, JavaScript)
    must_keep.update(re.findall(r'\b[A-Z][a-z]+(?:[A-Z][a-z]+)+\b', original))
    
    # Standard proper nouns (capitalized words not at sentence start)
    must_keep.update(re.findall(r'(?<!^)(?<!\. )[A-Z][a-z]+', original))
    
    # Tokens containing digits (o3, Llama 4, GPT-4, Section 504)
    must_keep.update(re.findall(r'\b\w*\d+\w*\b', original))
    
    # Quoted spans (preserve exact quotes)
    must_keep.update(re.findall(r'"[^"]+"', original))
    must_keep.update(re.findall(r"'[^']+'", original))
    
    # Filter out very short items that might be false positives
    must_keep = {t for t in must_keep if len(t) > 1}
    
    if not must_keep:
        return True
    
    # Check how many must-keep tokens appear in output
    missing = [t for t in must_keep if t not in text]
    missing_ratio = len(missing) / len(must_keep)
    return missing_ratio < 0.3  # Less than 30% missing

def check_numbers_preserved(text: str, original: str) -> bool:
    """Check if numbers and dates are preserved."""
    original_nums = set(re.findall(r'\b\d+(?:[.,]\d+)*\b', original))
    text_nums = set(re.findall(r'\b\d+(?:[.,]\d+)*\b', text))
    
    if not original_nums:
        return True
    missing_ratio = len(original_nums - text_nums) / len(original_nums)
    return missing_ratio < 0.2  # Less than 20% missing

def check_bullet_appropriateness(text: str, original: str) -> bool:
    """Check if bullets are used appropriately."""
    has_output_bullets = bool(re.search(r'[•\-\*]\s|^\d+\.\s', text, re.MULTILINE))
    
    # If output has bullets, check if source justifies them
    if has_output_bullets:
        # Check if original has multiple sentences or is a list
        sentence_count = len([s for s in re.split(r'[.!?\n]', original) if s.strip()])
        has_source_bullets = bool(re.search(r'[•\-\*]\s|^\d+\.\s', original, re.MULTILINE))
        
        return sentence_count >= 3 or has_source_bullets
    return True  # No bullets is always acceptable

def is_heading(text: str, element_tag: str = None) -> bool:
    """Check if text is a heading using tag metadata or heuristics."""
    # If we have element tag metadata, use it
    if element_tag and element_tag.lower() in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'title']:
        return True
    
    text = text.strip()
    
    # Heuristics for heading detection:
    # 1. Short text (< 15 words, < 150 chars)
    is_short = len(text.split()) <= 15 and len(text) < 150
    
    # 2. No period at end
    no_period = not text.endswith('.')
    
    # 3. No common verb forms (headings typically don't have verbs)
    has_common_verb = bool(re.search(r'\b(is|are|was|were|has|have|be|been|will|would|can|could|should|must)\b', text, re.IGNORECASE))
    
    return is_short and no_period and not has_common_verb

def has_bullets(text: str) -> bool:
    """Check if text contains bullet points."""
    return bool(re.search(r'[•\-\*]\s|^\d+\.\s', text, re.MULTILINE))

def categorize_tag(tag: str) -> str:
    """Categorize HTML element tag into chunk types."""
    tag = (tag or '').lower()
    if tag in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'title']:
        return 'heading'
    elif tag in ['p', 'div', 'span', 'article', 'section']:
        return 'paragraph'
    elif tag in ['li', 'ul', 'ol', 'dt', 'dd']:
        return 'list'
    elif tag in ['a', 'td', 'th', 'label', 'button', 'input']:
        return 'fragment'
    else:
        return 'other'

The following helper functions moved further up in the notebook because these functions are used by both the main evaluation and focused tests.

In [19]:
def render_prompt(template_source: str, text: str) -> str:
    """Render a Handlebars template with the given text."""
    compiler = Compiler()
    template = compiler.compile(template_source)
    return template({"text": text})

def call_groq(user_prompt: str, system_prompt: str = None, model: str = GROQ_MODEL, temperature: float = 0.1) -> str:
    """Call the Groq API with the given prompts."""
    try:
        messages = [{"role": "user", "content": user_prompt}]
        if system_prompt:
            messages.insert(0, {"role": "system", "content": system_prompt})
            
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=2000
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"[Error: {e}]"

print("✅ Helper functions defined (render_prompt, call_groq)")

✅ Helper functions defined (render_prompt, call_groq)


## 2.5 Debug Test - Verify TF-IDF Works

This cell tests that the `keep_meaning` rule correctly calculates TF-IDF similarity scores.

In [20]:
# DEBUG TEST: Verify TF-IDF and evaluate_rules are working correctly
print("🔍 Testing TF-IDF and evaluate_rules...\n")

test_original = "The quick brown fox jumps over the lazy dog."
test_simple = "A fast brown fox jumps over a lazy dog."

# Test 1: Direct TF-IDF call
similarity = tfidf_similarity(test_original, test_simple)
print(f"✓ Direct TF-IDF result: {similarity} (type: {type(similarity).__name__})")

# Test 2: Full evaluate_rules call
results = evaluate_rules(test_simple, test_original)
keep_meaning_result = results.get("keep_meaning", {})
print(f"✓ keep_meaning value: {keep_meaning_result.get('value')} (type: {type(keep_meaning_result.get('value')).__name__})")
print(f"✓ keep_meaning pass: {keep_meaning_result.get('pass')}")

# Test 3: Verify it's a float
if isinstance(keep_meaning_result.get('value'), float):
    print(f"\n✅ SUCCESS! TF-IDF is working correctly. Similarity: {keep_meaning_result.get('value'):.1%}")
else:
    print(f"\n❌ ERROR! Expected float, got {type(keep_meaning_result.get('value')).__name__}")

print("\n" + "="*60)

🔍 Testing TF-IDF and evaluate_rules...

✓ Direct TF-IDF result: 0.533 (type: float64)
✓ keep_meaning value: 0.533 (type: float64)
✓ keep_meaning pass: True

✅ SUCCESS! TF-IDF is working correctly. Similarity: 53.3%



## 3. Visualization Function

In [21]:
def display_side_by_side(original: str, results: dict, test_name: str):
    """Display model outputs side by side with rule evaluation."""
    
    n_models = len(results)
    
    # Header
    html = f"""<div style='background:#1a1a2e; padding:15px; border-radius:8px; margin:10px 0;'>
    <h3 style='color:#eee; margin:0 0 10px 0;'>📄 {test_name}</h3>
    <div style='background:#16213e; padding:10px; border-radius:5px; margin-bottom:15px;'>
        <strong style='color:#888;'>Original:</strong>
        <p style='color:#aaa; margin:5px 0; font-size:13px;'>{original[:300]}{'...' if len(original) > 300 else ''}</p>
    </div>
    <div style='display:flex; gap:10px;'>"""
    
    # Each model column
    for model_name, data in results.items():
        output = data.get("output", "")
        rules = data.get("rules", {})
        
        # Calculate rule score
        if rules:
            passed = sum(1 for r in rules.values() if r["pass"])
            total = len(rules)
            score_pct = (passed / total) * 100
            score_color = "#4ade80" if score_pct >= 80 else "#fbbf24" if score_pct >= 60 else "#f87171"
            score_html = f"<span style='background:{score_color}; color:#000; padding:2px 8px; border-radius:10px; font-size:12px;'>{passed}/{total} rules</span>"
        else:
            score_html = ""
        
        html += f"""
        <div style='flex:1; background:#0f3460; padding:12px; border-radius:6px;'>
            <div style='display:flex; justify-content:space-between; align-items:center; margin-bottom:10px;'>
                <strong style='color:#e0e0e0;'>{model_name}</strong>
                {score_html}
            </div>
            <div style='background:#1a1a2e; padding:10px; border-radius:4px; margin-bottom:10px; max-height:400px; overflow-y:auto;'>
                <pre style='color:#ddd; font-size:12px; white-space:pre-wrap; margin:0;'>{output}</pre>
            </div>
            <div style='font-size:11px;'>"""
        
        # Rule indicators
        for rule_id, result in rules.items():
            icon = "✅" if result["pass"] else "❌"
            if rule_id in EASY_LANGUAGE_RULES:
                rule_name = EASY_LANGUAGE_RULES[rule_id]["name"]
                
                # Show value for specific rules
                if rule_id == "short_sentences":
                    value_str = f" ({result['value']:.1f}% > 10w)"
                elif rule_id == "keep_meaning" and isinstance(result['value'], float):
                    value_str = f" ({result['value']:.0%})"
                else:
                    value_str = ""
                
                html += f"<div style='color:#aaa;'>{icon} {rule_name}{value_str}</div>"
        
        html += "</div></div>"
    
    html += "</div></div>"
    display(HTML(html))

## 4. Input Data

In [22]:
try:
    PROJECT_ROOT = Path(__file__).resolve().parents[1]
except NameError:
    # Fallback if __file__ is not defined (e.g. interactive mode)
    PROJECT_ROOT = Path(os.getcwd()).parent

SAMPLES_DIR = PROJECT_ROOT / "data" / "samples"

SAMPLE_CATEGORIES_EN = {
    "en_academic.txt": "Academic",
    "en_medical.txt": "Medical",
    "en_legal.txt": "Legal",
    "en_insurance.txt": "Insurance",
    "en_technical.txt": "Technical",
    "en_government.txt": "Government",
    "en_literature.txt": "Literature",
}

def get_all_samples_en() -> list[dict]:
    samples = []
    # Sort to ensure consistent order
    for filename in sorted(SAMPLE_CATEGORIES_EN.keys()):
        filepath = SAMPLES_DIR / filename
        if filepath.exists():
            text = filepath.read_text(encoding="utf-8").strip()
            samples.append({
                "filename": filename,
                "category": SAMPLE_CATEGORIES_EN[filename],
                "text": text
            })
    return samples

samples = get_all_samples_en()
print(f"Found {len(samples)} English samples in {SAMPLES_DIR}")

Found 7 English samples in /Users/esmahoney/Projects/klartext/klartext/data/samples


## 5. Prompt Definitions

In [ ]:
# Universal Parts
PROMPT_IDENTITY = """# Identity

You are an expert in plain language writing.
You specialize in rewriting text to be accessible 
to people with learning disabilities or low literacy.
"""

PROMPT_INSTRUCTIONS = """# Core Task 

* Rewrite the input text to be extremely simple and easy to understand.
* Keep the same meaning as the original text.

# Constraints

* Do NOT include any introductory or concluding text (e.g., "Here is the simplified text").
* Output ONLY the simplified text.
* Never output any XML/HTML tags or attributes (no <...>, no id=...).

# Structure & Formatting Rules

* Use clear structure.
* Use bullet points for steps, lists, or multiple items. Otherwise prefer short sentences.
* Add blank lines between every paragraph.
"""

EL_RULES_TEXT = """# Plain Language Rules
# Sentence & Length Rules

* Use very short sentences (maximum 10 words per sentence).
* Break up long sentences.
* Keep subjects and verbs close together.

# Vocabulary & Wording Rules

* Use simple, familiar words. Avoid technical, foreign, or formal terms.
* Explain any uncommon or necessary technical words or abbreviations in parentheses the first time they appear.
* Explain complex ideas or uncommon nouns in parentheses.
* Use positive wording. Avoid negations and never use double negatives.
* Replace abstract nouns with concrete, active verbs.

# Tone & Audience Rules

* Prefer active voice. Avoid passive voice whenever possible.
* Address the reader directly using “you”.
* Use a friendly, neutral tone.
* Avoid bureaucratic, legalistic, or commanding language.

# Consistency Rules

* Remove filler words and unnecessary details. Keep only essential information.
* Use the same words consistently. Do not switch terms for the same thing.
"""

PROMPT_EXAMPLES = """# Examples
# The following are example pairs.
# Learn the style and constraints from them.
# Do NOT copy the XML tags into your output.

<examples>

  <example id="1">
    <original_text>
    Upon arrival at the facility, visitors are required to sign in at the front desk and present valid photo identification.
    </original_text>

    <simplified_text>
    When you arrive:

    * Go to the front desk.
    * Sign in with your name.
    * Show your photo ID.
    </simplified_text>
  </example>

  <example id="2">
    <original_text>
    The medication should be administered twice daily with food to minimize potential gastrointestinal discomfort.
    </original_text>

    <simplified_text>
    Take this medicine two times every day.

    * Eat food when you take it. This helps your stomach feel better.
    </simplified_text>
  </example>

</examples>
"""

# ==========================================
# PROMPT A (BASELINE)
# ==========================================
PROMPT_A_SYSTEM = f"""{PROMPT_IDENTITY}

{PROMPT_INSTRUCTIONS}

{EL_RULES_TEXT}

{PROMPT_EXAMPLES}"""

PROMPT_USER_TEMPLATE = "Rewrite this text in simple language:\n{{text}}"

# ==========================================
# PROMPT B (EXPERIMENTAL)
# ==========================================

# You can modify these components to experiment
PROMPT_B_IDENTITY = PROMPT_IDENTITY

PROMPT_B_GOAL = """# Goal
Rewrite the input text to be extremely simple and easy to understand.
Keep the same meaning as the original text.
"""

PROMPT_B_INSTRUCTIONS = """# Core Task 

* Rewrite the input text to be extremely simple and easy to understand.
* Keep the same meaning as the original text.

# Non-negotiable constraints
* Do NOT add new information.
* Do NOT guess missing context.
* Do NOT explain or define terms unless the explanation already exists in the INPUT.
* Preserve all names, organizations, product names, model names, numbers, dates, and quoted phrases EXACTLY as written.
* Do NOT add words like “today”, “now”, or “currently” unless they appear in the INPUT.
* Output ONLY the rewritten text. No preface. No “here is…”. No commentary.
* Do not output any HTML/XML.

# Chunk rules
* If the INPUT is a fragment (title, heading, UI label, menu item, table cell, short phrase, date line):
* Rewrite ONLY the words present.
* Do NOT expand into explanations, summaries, or bullet lists.
* If it is a date/time string: output the same date/time (no added context).

If the INPUT is a paragraph or multiple sentences:
* Use short sentences.
* Keep the original meaning and details.
* Keep the original tone (do not turn facts into instructions).
* Use bullets ONLY when the source is already a list OR when there are 3+ distinct facts clearly stated in the INPUT.

# Style
* Prefer direct statements. Avoid “you/your” unless the INPUT is already addressing the reader.
* Use simple words.
* Add blank lines between paragraphs.

# Handling non-language / IDs
If the INPUT is mostly an ID, citation, DOI, ISBN, URL, or symbols:
* Output it unchanged.

# Structure & Formatting Rules
* Use clear structure.
* Use bullet points for steps, lists, or multiple items. Otherwise prefer short sentences.
* Add blank lines between every paragraph.
"""

EL_B_RULES_TEXT = """# Plain Language Rules
# Sentence & Length Rules

* Use very short sentences (maximum 10 words per sentence).
* Break up long sentences.
* Keep subjects and verbs close together.

# Vocabulary & Wording Rules

* Use simple, familiar words. Avoid technical, foreign, or formal terms.
* Explain any uncommon or necessary technical words or abbreviations in parentheses the first time they appear.
* Explain complex ideas or uncommon nouns in parentheses.
* Use positive wording. Avoid negations and never use double negatives.
* Replace abstract nouns with concrete, active verbs.

# Tone & Audience Rules

* Prefer active voice. Avoid passive voice whenever possible.
* Use a friendly, neutral tone.
* Avoid bureaucratic, legalistic, or commanding language.

# Consistency Rules

* Remove filler words and unnecessary details. Keep only essential information.
* Use the same words consistently. Do not switch terms for the same thing.
"""

PROMPT_B_EXAMPLES = """# Examples
# The following are example pairs.
# Learn the style and constraints from them.
# Do NOT copy the XML tags into your output.

<examples>

  <example id="1" type="single-sentence-to-prose">
    <original_text>
    The application must be submitted within 30 days of notification.
    </original_text>

    <simplified_text>
    Submit your application within 30 days.
    You start counting after you get the notice.
    </simplified_text>
  </example>

  <example id="2" type="multi-step-to-bullets">
    <original_text>
    Upon arrival at the facility, visitors are required to sign in at the front desk and present valid photo identification.
    </original_text>

    <simplified_text>
    When you arrive:

    * Go to the front desk.
    * Sign in.
    * Show your photo ID.
    </simplified_text>
  </example>

  <example id="3" type="heading-stays-short">
    <original_text>
    Policy-reserved domains
    </original_text>

    <simplified_text>
    Reserved domain names
    </simplified_text>
  </example>

  <example id="4" type="preserve-names-and-dates">
    <original_text>
    OpenAI released o3 and Llama 4 Scout on January 15, 2026.
    </original_text>

    <simplified_text>
    OpenAI released o3 and Llama 4 Scout.
    This happened on January 15, 2026.
    </simplified_text>
  </example>

</examples>
"""

PROMPT_B_RULES = """# Plain Language Rules
# Sentence & Length Rules

* Use very short sentences in the output (maximum 10 words per sentence).
* If a sentence is long, break it into multiple sentences.
* Keep subjects and verbs close together.

# Vocabulary & Wording Rules

* Use simple, familiar words. Avoid technical, foreign, or formal terms.
* Explain any uncommon or necessary technical words or abbreviations in parentheses the first time they appear.
* When a word is uncommon, explain the word in parentheses the first time it appears.
* Explain complex ideas or uncommon nouns in parentheses.
* Use positive wording. Avoid negations and never use double negatives.
* Replace abstract nouns with concrete, active verbs.

# Preservation Rules

* Preserve all proper nouns (people, organizations, products, models) exactly.
* Preserve all numbers and dates exactly as written.
* If the input text is incomplete or cut off, do not complete it. Note it's incomplete.

# Tone & Audience Rules

* Prefer active voice. Avoid passive voice whenever possible.
* Maintain conditional language when required.
* Write direct statements. Avoid "you/your" unless it's natural for the context.
* Use a friendly, neutral tone.
* Avoid bureaucratic, legalistic, or commanding language.

# Consistency Rules

* Remove filler words and unnecessary details. Keep only essential information.
* Do not explain ideas using the same language as the source.
* Use the same words consistently. Do not switch terms for the same thing.
"""

PROMPT_B_EXAMPLES = PROMPT_EXAMPLES

PROMPT_B_SYSTEM = f"""{PROMPT_B_IDENTITY}

{PROMPT_B_INSTRUCTIONS}

{PROMPT_B_RULES}

{PROMPT_B_EXAMPLES}"""

PROMPT_B_USER_TEMPLATE = PROMPT_USER_TEMPLATE


In [24]:
# Post-Generation Guardrails
# These functions check for common issues after text simplification

def check_meta_phrases(text: str) -> list[str]:
    """Detect unwanted meta phrases."""
    meta_phrases = [
        r"here'?s? the (rewritten|simplified|plain language)",
        r"you want to know",
        r"you are looking at",
        r"this (is|means)",
        r"the following (is|means)",
        r"in summary",
        r"to summarize"
    ]
    found = []
    for pattern in meta_phrases:
        if re.search(pattern, text, re.IGNORECASE):
            found.append(pattern)
    return found

def apply_guardrails(output: str, original: str) -> dict:
    """Apply post-generation checks and return warnings."""
    warnings = []
    
    # Check for meta phrases
    meta = check_meta_phrases(output)
    if meta:
        warnings.append(f"Contains meta phrases: {', '.join(meta)}")
    
    # Check number preservation
    orig_nums = set(re.findall(r'\b\d+(?:[.,]\d+)*\b', original))
    out_nums = set(re.findall(r'\b\d+(?:[.,]\d+)*\b', output))
    missing_nums = orig_nums - out_nums
    if missing_nums:
        warnings.append(f"Missing numbers: {', '.join(list(missing_nums)[:3])}")
    
    # Check for temporal injection
    temporal = ['today', 'now', 'currently']
    for word in temporal:
        if word in output.lower() and word not in original.lower():
            warnings.append(f"Injected temporal word: '{word}'")
    
    return {
        "passed": len(warnings) == 0,
        "warnings": warnings
    }

print("✅ Guardrail functions defined. These will be tested after evaluation runs.")

✅ Guardrail functions defined. These will be tested after evaluation runs.


In [25]:
#add focused test cases for known issues

FOCUSED_TEST_CASES = [
    {
        "name": "Heading Expansion Test",
        "text": "The Silent Shift in AI Development",
        "expected_issue": "Should stay as heading, not expand into bullets"
    },
    {
        "name": "Proper Noun Preservation",
        "text": "OpenAI's o3 and Llama 4 Scout are new AI models released in 2026.",
        "expected_issue": "Must preserve: OpenAI, o3, Llama 4 Scout, 2026"
    },
    {
        "name": "Date Handling",
        "text": "The meeting is scheduled for Tuesday, January 13, 2026.",
        "expected_issue": "Should not add 'today is' or 'currently'"
    },
    {
        "name": "Single Sentence",
        "text": "The application must be submitted within 30 days of notification.",
        "expected_issue": "Should be 1-2 sentences, not bulleted list"
    },
    {
        "name": "DOI/Citation",
        "text": "Reference: doi:10.1017/S0140525X00000000",
        "expected_issue": "Should pass through or handle gracefully, not refuse"
    }
]

print(f"\nAdded {len(FOCUSED_TEST_CASES)} focused test cases")
print("These will test specific issues from Improvement_Strategy.md")


Added 5 focused test cases
These will test specific issues from Improvement_Strategy.md


# Run Focused Tests

print("\n" + "="*60)
print("🎯 RUNNING FOCUSED TESTS")
print("="*60)

focused_results = []

for test_case in FOCUSED_TEST_CASES:
    print(f"\nTest: {test_case['name']}")
    print(f"Expected: {test_case['expected_issue']}")
    
    original_text = test_case["text"]
    
    # Run Prompt B only for focused tests
    prompt_b = render_prompt(PROMPT_B_USER_TEMPLATE, original_text)
    output_b = call_groq(prompt_b, system_prompt=PROMPT_B_SYSTEM)
    rules_b = evaluate_rules(output_b, original_text)
    guardrails_b = apply_guardrails(output_b, original_text)
    
    results_data = {
        "Prompt B (Experimental)": {
            "output": output_b,
            "rules": rules_b,
            "guardrails": guardrails_b
        }
    }
    
    focused_results.append({
        "test": test_case["name"],
        "original": original_text,
        "results": results_data
    })
    
    display_side_by_side(original_text, results_data, test_case['name'])
    
    # Display guardrail results
    if not guardrails_b["passed"]:
        print(f"⚠️  Guardrail warnings: {', '.join(guardrails_b['warnings'])}")
    
    time.sleep(1)

In [26]:
import json
import random
from collections import defaultdict

def load_benchmark_log(log_path: str) -> list[dict]:
    """Load benchmark log file (JSONL format)."""
    entries = []
    with open(log_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                try:
                    entries.append(json.loads(line))
                except json.JSONDecodeError:
                    continue
    return entries

def stratified_sample_from_logs(log_path: str, n_per_bucket: int = 3, seed: int = 42) -> list[dict]:
    """
    Sample originalText entries stratified by elementTag + length bucket.
    
    Args:
        log_path: Path to the benchmark log file
        n_per_bucket: Number of samples per (tag_type, length_bucket) combination
        seed: Random seed for reproducibility
        
    Returns:
        List of sample dictionaries with source_text, element_tag, page_url, etc.
    """
    random.seed(seed)
    entries = load_benchmark_log(log_path)
    
    # Filter to successful entries with source text
    valid_entries = [
        e for e in entries 
        if e.get('status') == 'success' and e.get('source_text')
    ]
    
    # Bucket by (tag_type, length_bucket)
    buckets = defaultdict(list)
    for entry in valid_entries:
        tag_type = categorize_tag(entry.get('element_tag', 'p'))
        text_len = len(entry.get('source_text', ''))
        
        if text_len < 20:
            length_bucket = 'short'
        elif text_len < 200:
            length_bucket = 'medium'
        else:
            length_bucket = 'long'
        
        buckets[(tag_type, length_bucket)].append(entry)
    
    # Sample from each bucket
    samples = []
    for (tag_type, length_bucket), items in buckets.items():
        n = min(n_per_bucket, len(items))
        if n > 0:
            sampled = random.sample(items, n)
            for item in sampled:
                samples.append({
                    'source_text': item['source_text'],
                    'element_tag': item.get('element_tag', 'p'),
                    'page_url': item.get('page_url', ''),
                    'tag_type': tag_type,
                    'length_bucket': length_bucket,
                    'prev_output': item.get('prev_output')  # If available for comparison
                })
    
    print(f"📊 Stratified sampling results:")
    print(f"   Total valid entries: {len(valid_entries)}")
    print(f"   Buckets found: {len(buckets)}")
    for key, items in sorted(buckets.items()):
        print(f"   - {key[0]:12} x {key[1]:8}: {len(items):4} entries")
    print(f"   Samples selected: {len(samples)}")
    
    return samples

# Test the stratified sampling if benchmark file exists
# Note: Benchmark files moved from apps/extension/logs/ to data/benchmarks/v1/
LOG_PATH = PROJECT_ROOT / "data" / "benchmarks" / "v1" / "klartext_benchmark_v1.json"

if LOG_PATH.exists():
    stratified_samples = stratified_sample_from_logs(str(LOG_PATH), n_per_bucket=2)
    print(f"\n✅ Loaded {len(stratified_samples)} stratified samples from benchmark")
else:
    stratified_samples = []
    print(f"⚠️ Benchmark file not found at {LOG_PATH}")

📊 Stratified sampling results:
   Total valid entries: 198
   Buckets found: 7
   - fragment     x medium  :   40 entries
   - heading      x medium  :   26 entries
   - list         x long    :    3 entries
   - list         x medium  :    6 entries
   - other        x medium  :    9 entries
   - paragraph    x long    :   34 entries
   - paragraph    x medium  :   80 entries
   Samples selected: 14

✅ Loaded 14 stratified samples from logs


In [27]:
# Run Focused Tests

print("\n" + "="*60)
print("🎯 RUNNING FOCUSED TESTS")
print("="*60)

focused_results = []

for test_case in FOCUSED_TEST_CASES:
    print(f"\nTest: {test_case['name']}")
    print(f"Expected: {test_case['expected_issue']}")
    
    original_text = test_case["text"]
    
    # Run Prompt B only for focused tests
    prompt_b = render_prompt(PROMPT_B_USER_TEMPLATE, original_text)
    output_b = call_groq(prompt_b, system_prompt=PROMPT_B_SYSTEM)
    rules_b = evaluate_rules(output_b, original_text)
    guardrails_b = apply_guardrails(output_b, original_text)
    
    results_data = {
        "Prompt B (Experimental)": {
            "output": output_b,
            "rules": rules_b,
            "guardrails": guardrails_b
        }
    }
    
    focused_results.append({
        "test": test_case["name"],
        "original": original_text,
        "results": results_data
    })
    
    display_side_by_side(original_text, results_data, test_case['name'])
    
    # Display guardrail results
    if not guardrails_b["passed"]:
        print(f"⚠️  Guardrail warnings: {', '.join(guardrails_b['warnings'])}")
    
    time.sleep(1)


🎯 RUNNING FOCUSED TESTS

Test: Heading Expansion Test
Expected: Should stay as heading, not expand into bullets



Test: Proper Noun Preservation
Expected: Must preserve: OpenAI, o3, Llama 4 Scout, 2026



Test: Date Handling
Expected: Should not add 'today is' or 'currently'



Test: Single Sentence
Expected: Should be 1-2 sentences, not bulleted list



Test: DOI/Citation
Expected: Should pass through or handle gracefully, not refuse


## 6. Run & Evaluate

In [28]:
# Main Evaluation Loop
# Note: render_prompt and call_groq are defined in section 5.5

print("Starting Evaluation Loop...")
results_history = []

for i, sample in enumerate(samples, 1):
    filename = sample["filename"]
    category = sample["category"]
    original_text = sample["text"]
    
    print(f"Processing {i}/{len(samples)}: {filename} ({category})...")
    
    # 1. Run Prompt A
    prompt_a = render_prompt(PROMPT_USER_TEMPLATE, original_text)
    output_a = call_groq(prompt_a, system_prompt=PROMPT_A_SYSTEM)
    rules_a = evaluate_rules(output_a, original_text)
    
    # 2. Run Prompt B
    prompt_b = render_prompt(PROMPT_B_USER_TEMPLATE, original_text)
    output_b = call_groq(prompt_b, system_prompt=PROMPT_B_SYSTEM)
    rules_b = evaluate_rules(output_b, original_text)
    
    # 3. Collect & Display
    results_data = {
        "Prompt A (Baseline)": {"output": output_a, "rules": rules_a},
        "Prompt B (Experimental)": {"output": output_b, "rules": rules_b}
    }
    
    results_history.append({
        "sample": filename,
        "results": results_data
    })
    
    display_side_by_side(original_text, results_data, f"{filename} ({category})")
    time.sleep(1) # Be nice to API

# --- Summary Section ---
print("\n" + "="*40)
print("🏁 FINAL SCORE SUMMARY")
print("="*40)

summary_html = """
<div style='background:#1a1a2e; padding:20px; border-radius:8px; margin-top:20px;'>
    <h2 style='color:#eee; border-bottom:1px solid #333; padding-bottom:10px;'>🏆 Final Evaluation Summary</h2>
    <table style='width:100%; border-collapse:collapse; color:#ddd;'>
        <tr style='background:#16213e; text-align:left;'>
            <th style='padding:10px;'>Model / Prompt</th>
            <th style='padding:10px;'>Total Rules Passed</th>
            <th style='padding:10px;'>Average Meaning</th>
            <th style='padding:10px;'>Pass Rate</th>
        </tr>
"""

models = ["Prompt A (Baseline)", "Prompt B (Experimental)"]
for model in models:
    total_passed = 0
    total_rules = 0
    total_meaning = 0.0
    count_meaning = 0
    
    for item in results_history:
        rules = item["results"][model]["rules"]
        total_passed += sum(1 for r in rules.values() if r["pass"])
        total_rules += len(rules)
        
        # Calculate meaning accuracy
        val = rules.get("keep_meaning", {}).get("value", 0)
        if isinstance(val, (int, float)):
             total_meaning += val
             count_meaning += 1
    
    avg_meaning_pct = (total_meaning / count_meaning * 100) if count_meaning else 0
    rule_pass_rate = (total_passed / total_rules * 100) if total_rules else 0
    
    summary_html += f"""
        <tr style='border-bottom:1px solid #333;'>
            <td style='padding:10px; font-weight:bold;'>{model}</td>
            <td style='padding:10px;'>{total_passed}/{total_rules}</td>
            <td style='padding:10px;'>{avg_meaning_pct:.1f}%</td>
            <td style='padding:10px;'>
                <div style='background:#333; width:100px; height:6px; border-radius:3px;'>
                    <div style='background:{'#4ade80' if rule_pass_rate >= 80 else '#fbbf24' if rule_pass_rate >= 60 else '#f87171'}; width:{rule_pass_rate}%; height:100%; border-radius:3px;'></div>
                </div>
            </td>
        </tr>
    """

summary_html += "</table></div>"
display(HTML(summary_html))

Starting Evaluation Loop...
Processing 1/7: en_academic.txt (Academic)...


Processing 2/7: en_government.txt (Government)...


Processing 3/7: en_insurance.txt (Insurance)...


Processing 4/7: en_legal.txt (Legal)...


Processing 5/7: en_literature.txt (Literature)...


Processing 6/7: en_medical.txt (Medical)...


Processing 7/7: en_technical.txt (Technical)...



🏁 FINAL SCORE SUMMARY


Model / Prompt,Total Rules Passed,Average Meaning,Pass Rate
Prompt A (Baseline),57/77,27.5%,
Prompt B (Experimental),60/77,32.0%,


# Summary

**Original prompt had performance of:**
| Model / Prompt | Total Rules Passed | Average Meaning | Pass Rate |
|---|---|---|---|
| Prompt A (Baseline) | 37/49 | 27.0% | |
| Prompt B (Experimental) | 38/49 | 30.3% | |


**v1 -**
initial runs with no rule adjustments - clearly something went wrong here
Updated prompt had performance of:
 Model / Prompt | Total Rules Passed | Average Meaning | Pass Rate |
|---|---|---|---|
| Prompt A (Baseline) | 71/84 | 100.0% | |	
| Prompt B (Experimental) |	70/84 |	100.0% | |	

These are more realistic but still I think something is off.
Model / Prompt | Total Rules Passed | Average Meaning | Pass Rate |
|---|---|---|---|
| Prompt A (Baseline) | 65/84 | 29.5%| |	
| Prompt B (Experimental) |	67/84|	31.2% | |	
		

Model / Prompt | Total Rules Passed | Average Meaning | Pass Rate |
|---|---|---|---|
| Prompt A (Baseline) | 66/84 | 29.6%| |	
| Prompt B (Experimental) |	65/84|	29.8% | |	

Found the error, pasted revised prompt into prompt A so it was comparing against same prompt, nice.





Well, these are disappointing results. Let's try a different approach.

1) evaluate and modify examples
Right now the samples may be contradicting the rules and need to be revised.
- example:
“For single sentences: output 1–2 short sentences (no bullets).”
But the example does the opposite: both are single-sentence originals and the outputs use bullets. This may be causing confusion.
- Fix:
Replace at least one example with a single sentence → 1–2 sentences, no bullets.
Keep a second example where bullets are justified (multi-sentence input or an input list).

- Pull examples from logs and also call the sample text from the templates.
- raw pool
- chunk log
- benchmark_v1.jasonl sample

2) Bullet behavior metric is not improving. But, the output looks pretty good. Maybe this is less about the ruleset and more about the reporting. Revise reporting to:
- Report per-rule pass rate and delta vs baseline, not just “total passed”.
- Report per chunk type (heading vs paragraph vs list) so bullets aren’t “failing” because the input was a nav label.

3) Like in first point, modfiy the examples and samples to better represent what's being seen in the logs.
- Take samples from the logs
- sample originalText entries stratified by elementTag + length bucket
- keep the category label as the page URL or site type
- run Prompt A and Prompt B on those originalText chunks

4) update preservve nouns, heading rules, intro check and active voice rules
a) now it looks like proper nouns rule won't capture things like acronyms, camelCase, tokens with digits, etc. To get a proper eval the rule can be updated to extract “must keep” tokens from the source and then require exact string containment using:
    - capitalized sequences including acronyms (2+ caps)
    - tokens containing digits (o3, Llama 4)
    - quoted spans
b) similarily with heading rule, too broad. Try changing from no_heading_expansion to is_heading() with the condition:
- headings have no verb (simple heuristic: no common verb forms / “is/are/was/were/has/have” for EN; for DE, skip this check or use chunk_type)
Note: ques: should elementTag be used?
c) Update intro text rule to check only the beginning.
- right now it looks like the re.match() may be picking up meta text and meta framing which can appear after a blank line or first sentence. Instead try changing to re.search() in the first 200 or so characters. 
Note: active voice metric is misleading. seems that this passes almost everytime and I question if this is helpful or not. I kinda like it but not sure if it adds any value. 




In [29]:
def generate_per_rule_delta_report(results_history: list, model_a: str = "Prompt A (Baseline)", model_b: str = "Prompt B (Experimental)") -> pd.DataFrame:
    """
    Generate per-rule pass rate and delta report between two prompts.
    
    Args:
        results_history: List of result dictionaries from evaluation
        model_a: Name of baseline model
        model_b: Name of experimental model
        
    Returns:
        DataFrame with per-rule comparison
    """
    rules = list(EASY_LANGUAGE_RULES.keys())
    n_samples = len(results_history)
    
    report_data = []
    for rule in rules:
        pass_a = sum(1 for r in results_history if r['results'][model_a]['rules'].get(rule, {}).get('pass', False))
        pass_b = sum(1 for r in results_history if r['results'][model_b]['rules'].get(rule, {}).get('pass', False))
        
        rate_a = pass_a / n_samples * 100 if n_samples > 0 else 0
        rate_b = pass_b / n_samples * 100 if n_samples > 0 else 0
        delta = rate_b - rate_a
        
        # Determine trend
        if delta > 5:
            trend = "📈 Improved"
        elif delta < -5:
            trend = "📉 Regressed"
        else:
            trend = "➡️ Same"
        
        report_data.append({
            'Rule': EASY_LANGUAGE_RULES[rule]['name'],
            'Baseline': f"{pass_a}/{n_samples} ({rate_a:.0f}%)",
            'Experimental': f"{pass_b}/{n_samples} ({rate_b:.0f}%)",
            'Delta': f"{delta:+.1f}%",
            'Trend': trend
        })
    
    return pd.DataFrame(report_data)

def generate_chunk_type_report(results_history: list, element_tags: list = None) -> pd.DataFrame:
    """
    Generate per-chunk-type breakdown of rule pass rates.
    
    Args:
        results_history: List of result dictionaries
        element_tags: List of element tags corresponding to each result (optional)
        
    Returns:
        DataFrame with per-chunk-type breakdown
    """
    if not element_tags:
        # If no tags provided, show overall stats
        return None
    
    # Group results by chunk type
    chunk_groups = defaultdict(list)
    for i, (result, tag) in enumerate(zip(results_history, element_tags)):
        chunk_type = categorize_tag(tag)
        chunk_groups[chunk_type].append(result)
    
    report_data = []
    for chunk_type, items in sorted(chunk_groups.items()):
        n = len(items)
        
        # Calculate pass rates for key rules per chunk type
        bullet_passes = sum(1 for r in items if r['results']['Prompt B (Experimental)']['rules'].get('appropriate_bullet_use', {}).get('pass', False))
        heading_passes = sum(1 for r in items if r['results']['Prompt B (Experimental)']['rules'].get('no_heading_expansion', {}).get('pass', False))
        meaning_passes = sum(1 for r in items if r['results']['Prompt B (Experimental)']['rules'].get('keep_meaning', {}).get('pass', False))
        
        report_data.append({
            'Chunk Type': chunk_type,
            'Count': n,
            'Bullet Rule Pass': f"{bullet_passes}/{n} ({bullet_passes/n*100:.0f}%)" if n > 0 else "N/A",
            'Heading Rule Pass': f"{heading_passes}/{n} ({heading_passes/n*100:.0f}%)" if n > 0 else "N/A",
            'Meaning Pass': f"{meaning_passes}/{n} ({meaning_passes/n*100:.0f}%)" if n > 0 else "N/A"
        })
    
    return pd.DataFrame(report_data)

def display_enhanced_report(results_history: list, element_tags: list = None):
    """Display enhanced reporting with per-rule delta and chunk-type breakdown."""
    
    print("\n" + "="*60)
    print("📊 PER-RULE DELTA REPORT")
    print("="*60)
    
    df_rules = generate_per_rule_delta_report(results_history)
    display(df_rules)
    
    # Highlight improvements and regressions
    improvements = df_rules[df_rules['Delta'].str.contains(r'\+[5-9]|\+[1-9]\d', regex=True)]
    regressions = df_rules[df_rules['Delta'].str.contains(r'-[5-9]|-[1-9]\d', regex=True)]
    
    if len(improvements) > 0:
        print(f"\n✅ Rules that improved (>5%): {', '.join(improvements['Rule'].tolist())}")
    if len(regressions) > 0:
        print(f"\n⚠️ Rules that regressed (>5%): {', '.join(regressions['Rule'].tolist())}")
    
    if element_tags:
        print("\n" + "="*60)
        print("📋 PER-CHUNK-TYPE BREAKDOWN")
        print("="*60)
        
        df_chunks = generate_chunk_type_report(results_history, element_tags)
        if df_chunks is not None:
            display(df_chunks)

# Run enhanced report on results if available
if 'results_history' in dir() and len(results_history) > 0:
    display_enhanced_report(results_history)
    print("\n✅ Enhanced reporting functions available. Call display_enhanced_report(results_history, element_tags) after evaluation.")


📊 PER-RULE DELTA REPORT


,Rule,Baseline,Experimental,Delta,Trend
0,Short Sentences,1/7 (14%),0/7 (0%),-14.3%,📉 Regressed
1,Appropriate Bullet Points,4/7 (57%),6/7 (86%),+28.6%,📈 Improved
2,Clear Paragraphs,7/7 (100%),7/7 (100%),+0.0%,➡️ Same
3,No Intro/Outro Text,7/7 (100%),7/7 (100%),+0.0%,➡️ Same
4,No XML/HTML Tags,7/7 (100%),7/7 (100%),+0.0%,➡️ Same
5,Keep Meaning,1/7 (14%),3/7 (43%),+28.6%,📈 Improved
6,Active Voice,6/7 (86%),6/7 (86%),+0.0%,➡️ Same
7,Preserves Proper Nouns,4/7 (57%),4/7 (57%),+0.0%,➡️ Same
8,No Temporal Injection,7/7 (100%),7/7 (100%),+0.0%,➡️ Same
9,Preserves Numbers & Dates,6/7 (86%),6/7 (86%),+0.0%,➡️ Same



✅ Rules that improved (>5%): Appropriate Bullet Points, Keep Meaning

⚠️ Rules that regressed (>5%): Short Sentences

✅ Enhanced reporting functions available. Call display_enhanced_report(results_history, element_tags) after evaluation.


In [31]:
# Run evaluation on stratified samples (set RUN_STRATIFIED = True to execute)
RUN_STRATIFIED = True  # Change to True to run

if RUN_STRATIFIED and len(stratified_samples) > 0:
    print("\n" + "="*60)
    print("🔬 RUNNING EVALUATION ON STRATIFIED SAMPLES")
    print("="*60)
    
    stratified_results = []
    stratified_tags = []
    
    for i, sample in enumerate(stratified_samples[:20], 1):  # Limit to 20 for API rate limits
        original_text = sample['source_text']
        element_tag = sample['element_tag']
        tag_type = sample['tag_type']
        length_bucket = sample['length_bucket']
        
        print(f"\nProcessing {i}/{min(20, len(stratified_samples))}: [{tag_type}|{length_bucket}] {original_text[:50]}...")
        
        # Run Prompt A
        prompt_a = render_prompt(PROMPT_USER_TEMPLATE, original_text)
        output_a = call_groq(prompt_a, system_prompt=PROMPT_A_SYSTEM)
        rules_a = evaluate_rules(output_a, original_text)
        
        # Run Prompt B
        prompt_b = render_prompt(PROMPT_B_USER_TEMPLATE, original_text)
        output_b = call_groq(prompt_b, system_prompt=PROMPT_B_SYSTEM)
        rules_b = evaluate_rules(output_b, original_text)
        
        results_data = {
            "Prompt A (Baseline)": {"output": output_a, "rules": rules_a},
            "Prompt B (Experimental)": {"output": output_b, "rules": rules_b}
        }
        
        stratified_results.append({
            "sample": f"{tag_type}_{length_bucket}_{i}",
            "results": results_data,
            "element_tag": element_tag,
            "source_url": sample.get('page_url', '')
        })
        stratified_tags.append(element_tag)
        
        display_side_by_side(original_text, results_data, f"[{tag_type}|{length_bucket}]")
        time.sleep(1.5)  # Be nice to API
    
    # Display enhanced report for stratified samples
    print("\n" + "="*60)
    print("📊 STRATIFIED SAMPLES ANALYSIS")
    print("="*60)
    display_enhanced_report(stratified_results, stratified_tags)
    
else:
    if not RUN_STRATIFIED:
        print("⏭️ Skipping stratified sample evaluation. Set RUN_STRATIFIED = True to run.")
    else:
        print("⚠️ No stratified samples available. Check log file path.")


🔬 RUNNING EVALUATION ON STRATIFIED SAMPLES

Processing 1/14: [paragraph|medium] And then you get Genitiv and Dativ. Fun.......



Processing 2/14: [paragraph|medium] Central African Republic......



Processing 3/14: [fragment|medium] Research and statistics......



Processing 4/14: [fragment|medium] Expressions & operators......



Processing 5/14: [heading|medium] Set your community flair according to nationality ...


...


Processing 14/14: [paragraph|long] This shift wasn't just technical; it was psycholog...



📊 STRATIFIED SAMPLES ANALYSIS

📊 PER-RULE DELTA REPORT


,Rule,Baseline,Experimental,Delta,Trend
0,Short Sentences,7/14 (50%),9/14 (64%),+14.3%,📈 Improved
1,Appropriate Bullet Points,4/14 (29%),13/14 (93%),+64.3%,📈 Improved
2,Clear Paragraphs,12/14 (86%),4/14 (29%),-57.1%,📉 Regressed
3,No Intro/Outro Text,12/14 (86%),12/14 (86%),+0.0%,➡️ Same
4,No XML/HTML Tags,14/14 (100%),14/14 (100%),+0.0%,➡️ Same
5,Keep Meaning,8/14 (57%),12/14 (86%),+28.6%,📈 Improved
6,Active Voice,13/14 (93%),14/14 (100%),+7.1%,📈 Improved
7,Preserves Proper Nouns,8/14 (57%),8/14 (57%),+0.0%,➡️ Same
8,No Temporal Injection,14/14 (100%),14/14 (100%),+0.0%,➡️ Same
9,Preserves Numbers & Dates,14/14 (100%),13/14 (93%),-7.1%,📉 Regressed



✅ Rules that improved (>5%): Short Sentences, Appropriate Bullet Points, Keep Meaning, Active Voice

⚠️ Rules that regressed (>5%): Clear Paragraphs, Preserves Numbers & Dates

📋 PER-CHUNK-TYPE BREAKDOWN


,Chunk Type,Count,Bullet Rule Pass,Heading Rule Pass,Meaning Pass
0,fragment,2,2/2 (100%),2/2 (100%),2/2 (100%)
1,heading,2,1/2 (50%),2/2 (100%),0/2 (0%)
2,list,4,4/4 (100%),4/4 (100%),4/4 (100%)
3,other,2,2/2 (100%),2/2 (100%),2/2 (100%)
4,paragraph,4,4/4 (100%),4/4 (100%),4/4 (100%)
